In [1]:
import weaviate

from llama_index import StorageContext, SimpleDirectoryReader, ServiceContext, VectorStoreIndex, Document, get_response_synthesizer
from llama_index.vector_stores import WeaviateVectorStore
from llama_index.embeddings import LangchainEmbedding, OllamaEmbedding
from llama_index.indices.document_summary import DocumentSummaryIndex
from llama_index.llms import Ollama
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.postprocessor import SimilarityPostprocessor
from llama_index.retrievers import VectorIndexRetriever


import box
import yaml
import warnings

import logger as log

In [2]:
# logger = log.init_logger(__name__)
# logger.debug("start")


In [3]:
config_path='./.venv/config.yml'
with open(config_path, 'r', encoding='utf8') as ymlfile:
    cfg = box.Box(yaml.safe_load(ymlfile))

In [4]:
client = weaviate.Client(cfg.WEAVIATE_URL)

In [5]:
llm = Ollama(
            model=cfg.LLM,
            base_url=cfg.OLLAMA_BASE_URL,
            temperature=cfg.TEMPERATURE
        )

In [6]:
embeddings = OllamaEmbedding(model_name=cfg.LLM)

In [7]:
service_context = ServiceContext.from_defaults(
    embed_model=embeddings,
    llm=llm
)    

In [8]:
index_name = "A1148"

In [9]:
vector_store = WeaviateVectorStore(
    weaviate_client=client,
    # index_name=cfg.INDEX_NAME
    index_name = index_name
)

In [10]:
storage_context = StorageContext.from_defaults(
    vector_store=vector_store
) 

In [11]:
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
    use_async=False,
    service_context=service_context
)    

In [12]:
from llama_index import VectorStoreIndex

In [13]:
index_main = VectorStoreIndex.from_vector_store(
    vector_store = vector_store,
    service_context = service_context
)

In [14]:
retriever = VectorIndexRetriever(
    index=index_main,
    similarity_top_k=4,
) 

In [15]:
# zulip_query_engine = index_main.as_query_engine()
zulip_query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.3)]
)

In [16]:
response = zulip_query_engine.query("What are the imlications of scheduling a brainstorm about partners")
print(response)

The implications of scheduling a brainstorm about partners are:

1. Efficient organization: Scheduling a brainstorm specifically for partnership projects/activities can help ensure that these tasks are efficiently organized and prioritized within the Marketing meeting. This can help avoid confusion or overlap with other marketing activities.
2. Clear communication: By dedicating a separate meeting to partnership projects/activities, it ensures that there is clear communication and focus on these tasks without distractions from other marketing areas.
3. Better categorization: Using tags such as `aspect/marketing` and `product/balenaOS` can help categorize tasks more effectively within Fibery, making it easier to filter and organize them.
4. Improved navigation: Having a convention for naming streams and tags can help people navigate and find the information they need more efficiently.
5. Handling high-touch customers: By dedicating a separate meeting to partnership projects/activities, 

In [17]:
# for node in response.source_nodes:
#     print(node.node_id, node.score, node.get_text())

In [18]:
#### sample code to load a document into the current index (index_main)
with open("data/_announcements/(Fibery) Brainstorm calls migration", 'r') as file:
    text = " ".join(line.rstrip() for line in file)
document = Document(text=text, doc_id="brainstorm2", metadata={"stream": "_announcements", "doc_name": "brainstorm2"})
documents=[document]
index_main.insert(document, show_progress=True)

/home/rafa/Workspace/my-projects/zulip_summaries/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings:   0%|          | 0/8 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████| 8/8 [00:04<00:00,  1.99it/s]


In [19]:
doc_summary_index = DocumentSummaryIndex.from_documents(
    documents=documents,
    storage_context=storage_context,
    service_context=service_context,
    response_synthesizer=response_synthesizer,
    show_progress=True
   )

Summarizing documents:   0%|          | 0/1 [00:00<?, ?it/s]

current doc id: brainstorm2


Generating embeddings: 100%|██████████| 1/1 [00:01<00:00,  1.65s/it]


In [20]:
doc_summary_index.storage_context.persist(persist_dir="./storage")

In [21]:
from llama_index.indices.loading import load_index_from_storage
doc_summary_index2 = load_index_from_storage(storage_context=storage_context, service_context=service_context)

In [29]:
print(doc_summary_index2.get_document_summary("brainstorm2"))

The provided text is about a discussion on how to organize and track brainstorm calls and patterns in Fibery, specifically related to the marketing aspect. The participants are trying to understand why there is a separate meeting for partnership projects/activities instead of discussing everything under a Marketing meeting, and they mention the need to add meta-data to tasks in Fibery using tags such as `aspect/marketing` and `product/balenaOS` to categorize them. The text also touches upon the integration of events from Google Calendar into Fibery and how it relates to brainstorm calls.

Some of the questions that this text can answer include:

* Why do we have a separate meeting for partnership projects/activities instead of discussing everything under a Marketing meeting?
* How can we efficiently organize brainstorm calls in Fibery, especially related to marketing aspects?
* What is the best way to categorize tasks in Fibery to facilitate filtering and organization?
* How can we use

In [30]:
zulip_query_engine = doc_summary_index2.as_query_engine()


In [31]:
r = zulip_query_engine.query("is there a convention for naming streams")

KeyError: '8c2ece97-6598-4ee6-aa03-a173853b671c'

### Generating a summary of an existing stored document using it's title
I haven't been able to retreive the summary index. The index is not working. I've tried also persist in storage_context(storage_dir="./storage"), but this is not storing or loading any indexes.
Shame...

# doc_summary_index = DocumentSummaryIndex(service_context=service_context,
from llama_index.indices.document_summary import DocumentSummaryIndexLLMRetriever
from llama_index.indices.loading import load_index_from_storage, load_indices_from_storage
doc_summary_index = load_indices_from_storage(storage_context=storage_context, index_id=index_name)

In [ ]:
from llama_index.agent import 